In [134]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [135]:
# Define the grid world dimensions and number of gus
GRID_SIZE = 100
NUM_USERS = 13

# Q-learning parameters
MAX_STEPS = 1
LEARNING_RATE = 0.1
DISCOUNT_FACTOR = 0.99

# Exploration parameters
EPSILON_VALUES = [0.8, 0.6, 0.4, 0.3, 0.2, 0.15, 0.1, 0.05, 0.03, 0.01]

In [136]:
# Initialize grid world environments
def initialize_env():
    env = np.random.randint(0, GRID_SIZE, size=(NUM_USERS, 2))
    print("Initial Gus Positions:")
    print(env)
    return env

In [137]:
# Initial search route for reference
INITIAL_SEARCH_ROUTE = [
    [15,5],
    [15,25],
    [15,45],
    [15,65],
    [15,85],
    [35,85],
    [55,85],
    [75,85],
    [95,85],
    [95,65],
    [95,45],
    [95,25],
    [95,5],
    [75,5],
    [55,5],
    [35,5],
    [35,25],
    [35,45],
    [35,65],
    [55,65],
    [75,65],
    [75,45],
    [75,25],
    [55,25],
    [55,45]
]

In [138]:
# Check if a gu is found within the search radius
def is_gu_within_radius(gu_positions, state):
    num_gus_within_radius = 0
    for gu in gu_positions:
        distance = np.sqrt((state[0] - gu[0]) ** 2 + (state[1] - gu[1]) ** 2)
        if distance <= 17:  # Check if the distance is less than or equal to 17
            num_gus_within_radius += 1
    return num_gus_within_radius

In [139]:
# Check if a gu is found within the search radius
def is_gu_found(state, env):
    gus_found = 0
    for gu in env:
        if is_gu_within_radius(gu, state):
            gus_found += 1
    return gus_found

In [140]:
# Calculate reward based on the next state and environment
def get_reward(state, env):
    reward = -1  # Default reward for movement
    gus_in_radius = is_gu_within_radius(env, state)
    env = [gu for gu in env if not is_gu_within_radius([gu], state)]  # Remove gus within radius

    if gus_in_radius > 0:
        reward += 10 * gus_in_radius  # Plus reward for finding gus within the radius

    return reward, env, gus_in_radius

In [141]:
# Check if all gus are collected
def all_gus_collected(env):
    return len(env) == 0

In [142]:
# Get the next state based on current state and action
def get_next_state(state, action):
    if action == 0:  # Move up
        return max(state[0] - 1, 0), state[1]
    elif action == 1:  # Move down
        return min(state[0] + 1, GRID_SIZE - 1), state[1]
    elif action == 2:  # Move left
        return state[0], max(state[1] - 1, 0)
    else:  # Move right
        return state[0], min(state[1] + 1, GRID_SIZE - 1)

In [143]:
# Generate sequential search positions covering the entire grid
def generate_search_positions():
    positions = []
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            positions.append([i, j])
    return positions

In [144]:
# Q-learning using NumPy with searcher's movement and search radius
def q_learning(env, epsilon, reference_route):
    q_table = np.zeros((GRID_SIZE, GRID_SIZE, 4))  # Three dimensions: x, y, and actions
    rewards = []

    search_route = reference_route.copy()  # Copy the initial search route
    current_index = 0  # Index to keep track of the current position in search_route
    visited_points = set()  # Set to store visited points

    total_gus_found = 0  # Counter for total gus found across all episodes
    total_reward = 0
    gus_found = 0  # Counter for gus found in this episode

    for step in range(MAX_STEPS):
        state = tuple(search_route[0])  # Convert to tuple

        if state not in visited_points:
            visited_points.add(state)

            for _ in range(20):  # Move 20 units at each step
                state_int = (int(state[0]), int(state[1]))  # Convert to integer tuple for accessing q_table
                action = np.argmax(q_table[state_int[0], state_int[1], :])  # Exploit learned values
                next_state = get_next_state(list(state_int), action)  # Convert to list for updating

                next_state_int = (int(next_state[0]), int(next_state[1]))  # Convert to integer tuple

                reward, env, gus_in_radius = get_reward(next_state, env)  # Update env after gu found

                # Update q_table using indexing
                q_table[state_int[0], state_int[1], action] += LEARNING_RATE * (
                    reward + DISCOUNT_FACTOR ** (_ + 1) * np.max(q_table[next_state_int[0], next_state_int[1], :])
                    - q_table[state_int[0], state_int[1], action])

                total_reward += reward
                total_gus_found += gus_in_radius  # Accumulate gus found within radius

                if all_gus_collected(env):
                    break

                state = tuple(next_state_int)  # Convert back to tuple for dictionary key

                if gus_in_radius > 0:
                    env = [gu for gu in env if not is_gu_within_radius([gu], next_state)]  # Remove gus within radius

            # Rotate the search route
            search_route.append(search_route.pop(0))

            if all_gus_collected(env):
                break

        rewards.append(total_reward)
        total_gus_found += gus_found  # Accumulate gus found across episodes

    return q_table, rewards, total_gus_found

In [145]:
# Visualize the learned route
def visualize_route(q_table, env):
    route = []
    state = env[0]

    while not all_gus_collected(env):
        action = np.argmax(q_table[state[0], state[1], :])
        next_state = get_next_state(state, action)
        route.append(next_state)
        state = next_state
        if tuple(state) in env.tolist():
            env = np.delete(env, np.where((env == state).all(axis=1))[0][0], axis=0)

    route = np.array(route)
    sns.set_style("whitegrid")
    plt.figure(figsize=(10, 10))

    # Plot search route and gus positions
    sns.scatterplot(x=route[:, 1], y=route[:, 0], color='blue', s=50, label='Search Route')
    sns.scatterplot(x=env[:, 1], y=env[:, 0], color='red', marker='x', s=100, label='Gus Positions')

    # Plot search range circles around gus positions
    for gu in env:
        circle = plt.Circle((gu[1], gu[0]), radius=17, color='green', fill=False, linestyle='dashed')
        plt.gca().add_patch(circle)

    plt.title("Search Route Visualization")
    plt.xlabel("X Position")
    plt.ylabel("Y Position")
    plt.xlim(0, GRID_SIZE)
    plt.ylim(0, GRID_SIZE)
    plt.gca().invert_yaxis()
    plt.legend()
    plt.show()

In [146]:
for epsilon in EPSILON_VALUES:
    env = initialize_env()
    q_table, rewards, gus_found = q_learning(env, epsilon, INITIAL_SEARCH_ROUTE)

    total_reward = sum(rewards)
    print(f"Epsilon: {epsilon}, gus Found: {gus_found}, Total Reward: {total_reward}")

    #visualize_route(q_table, env)

Initial Gus Positions:
[[38 91]
 [25 95]
 [65 32]
 [68 80]
 [95 74]
 [46  4]
 [92 27]
 [89 61]
 [24  8]
 [16 15]
 [33 69]
 [ 2 63]
 [86 76]]
Epsilon: 0.8, gus Found: 2, Total Reward: 0
Initial Gus Positions:
[[62 82]
 [36 42]
 [75 69]
 [83 92]
 [66 16]
 [23 63]
 [ 0 64]
 [84 95]
 [60 11]
 [67 41]
 [86  1]
 [65 76]
 [21 66]]
Epsilon: 0.6, gus Found: 0, Total Reward: -20
Initial Gus Positions:
[[ 8 98]
 [36 29]
 [67 14]
 [41 55]
 [23 47]
 [43 70]
 [ 7 20]
 [73 99]
 [67 46]
 [10 37]
 [ 8 80]
 [23 62]
 [96 73]]
Epsilon: 0.4, gus Found: 1, Total Reward: -10
Initial Gus Positions:
[[99 26]
 [29  2]
 [62  0]
 [55  9]
 [31 62]
 [83 81]
 [30 50]
 [ 8 86]
 [28 42]
 [21 50]
 [22 97]
 [30  0]
 [79 28]]
Epsilon: 0.3, gus Found: 2, Total Reward: 0
Initial Gus Positions:
[[30 61]
 [19 37]
 [30  2]
 [31 43]
 [ 6 83]
 [84 32]
 [65 43]
 [75 16]
 [88 28]
 [67 93]
 [45 93]
 [84 47]
 [29 15]]
Epsilon: 0.2, gus Found: 1, Total Reward: -10
Initial Gus Positions:
[[29 81]
 [60 50]
 [62 78]
 [94 42]
 [84 89]
 